<a href="https://colab.research.google.com/github/BikmetovDanil/ML/blob/master/daciyk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Интеллектуальная исследовательская работа, робот Сергей Аркадьевич Дацюк
(сокращенно RoSD)

![link](https://i.ytimg.com/vi/gmlW0AzPjOQ/maxresdefault.jpg)

## deps

In [ ]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .
pip install datasets evaluate
pip install accelerate -U -qq

mkdir models/
wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py

## dataset

In [ ]:
dataset = ""
books = ["daciyk_konstructivizm.txt", "daciyk_ontologia.txt", "daciyk_snm.txt", "daciyk_perspectiva.txt"]
for book in books:
    with open(book) as f:
        dataset += f.read()
print(len(dataset))
with open("daciyk_all.txt", "w") as f:
    f.write(dataset)

In [ ]:
dataset[:300]

## train

In [ ]:
%load_ext tensorboard
%tensorboard --logdir models/essays/runs

In [ ]:
!python run_clm.py \
   --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
   --train_file daciyk_all.txt \
   --per_device_train_batch_size 12 \
   --block_size 1024 \
   --dataset_config_name plain_text \
   --do_train \
   --gradient_accumulation_steps 4 \
   --gradient_checkpointing True \
   --fp16 True \
   --optim adafactor \
   --num_train_epochs 7 \
   --output_dir models/essays \
   --overwrite_output_dir

## generate

In [ ]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


np.random.seed(42)
torch.manual_seed(42)

tokenizer = GPT2Tokenizer.from_pretrained("models/daciyk")
model = GPT2LMHeadModel.from_pretrained("models/daciyk")


model.cuda()

In [ ]:
inp_text = input("Prompt: ")

while inp_text != "":
    inpt = tokenizer.encode(inp_text, return_tensors="pt")
    out = model.generate(inpt.cuda(),
                        max_length=500,
                        repetition_penalty=6.0,
                        do_sample=True,
                        top_k=5,
                        top_p=0.95,
                        temperature=1,
                        no_repeat_ngram_size=2)
    print("RoSD:", tokenizer.decode(out[0]))
    inp_text = input("Prompt: ")